In [ ]:
# https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-python-sdk/scikit_learn_inference_pipeline

In [13]:
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.sklearn import SKLearnModel
from sagemaker.xgboost import XGBoostModel

sklearn_model = SKLearnModel(
    model_data=None,  # Not needed because this is a script
    role=sagemaker.get_execution_role(),
    entry_point='0_sklearn_inference.py',
    framework_version='0.23-1',  # Ensure this matches your SageMaker SKLearn container version
    sagemaker_session=sagemaker.Session()
)

xgboost_model = XGBoostModel(
   model_data='s3://ktzouvan-trading-point-sagemaker-poc/models/pipelines-3ol04e0v16ml-training-step-sALSc8gqXi/output/model.tar.gz',
   entry_point='0_xgboost_inference.py',
   framework_version='1.7-1',
   role=sagemaker.get_execution_role(),
)

In [ ]:
from sagemaker.workflow.model_step import ModelStep
from sagemaker import PipelineModel

pipeline_model = PipelineModel(
   models=[sklearn_model, xgboost_model],
   role=role,sagemaker_session=pipeline_session,
)

register_model_step_args = pipeline_model.register(
   content_types=["application/json"],
   response_types=["application/json"],
   inference_instances=["ml.m5.large"],
   transform_instances=["ml.m5.large"],
   model_package_group_name='container-two',